In [6]:
import numpy as np

def split_curve(curve, break_index):
    if 0 < break_index < len(curve) - 1:
        return curve[:break_index + 1], curve[break_index + 1:]
    elif break_index == len(curve) - 1:
        return curve[:break_index + 1], []
    else:
        return curve, []

def calculate_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def calculate_curvature(curve, interval=1.0):
    selected_points = [curve[0]]
    total_distance = 0.0

    for i in range(1, len(curve)):
        distance = calculate_distance(curve[i], selected_points[-1])
        total_distance += distance
        
        if total_distance >= interval:
            selected_points.append(curve[i])
            total_distance = 0.0
    
    curvatures = []
    for i in range(1, len(selected_points) - 1):
        prev_point = selected_points[i - 1]
        curr_point = selected_points[i]
        next_point = selected_points[i + 1]
        
        vector1 = np.array(curr_point) - np.array(prev_point)
        vector2 = np.array(next_point) - np.array(curr_point)
        
        angle = np.arccos(np.clip(np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)), -1.0, 1.0))
        
        curvature = angle / (np.linalg.norm(vector1) + np.linalg.norm(vector2))
        curvatures.append(curvature)
    
    return [0] + curvatures + [0]

def find_curvature_break_points(curve, curvature_threshold=0.2, exclusion_distance=10):
    curvatures = calculate_curvature(curve)
    break_points = []
    excluded_indices = set()
    
    for i, curvature in enumerate(curvatures):
        if curvature > curvature_threshold and i not in excluded_indices:
            break_points.append((i, curve[i]))
            
            for j in range(len(curve)):
                if np.linalg.norm(np.array(curve[i]) - np.array(curve[j])) < exclusion_distance:
                    excluded_indices.add(j)
    
    return break_points

def split_curve_by_curvature(curve, curvature_threshold=0.2, exclusion_distance=10, min_length=8, distance_threshold=30):
    if len(curve) == 0:
        return []  # Return an empty list if the curve is empty
    
    def curve_length(curve):
        length = 0.0
        for i in range(1, len(curve)):
            length += calculate_distance(curve[i - 1], curve[i])
        return length
    
    break_points = find_curvature_break_points(curve, curvature_threshold, exclusion_distance)
    break_indices = [bp[0] for bp in break_points]

    break_indices = sorted(set(break_indices))
    
    new_curves = []
    current_curve = curve.copy()
    
    for break_index in break_indices:
        if break_index >= len(current_curve):
            continue
        
        new_curve, remaining_curve = split_curve(current_curve, break_index)
        
        if curve_length(new_curve) < min_length or curve_length(remaining_curve) < min_length:
            continue  # Skip splitting at this break point
        
        if len(new_curve) > 0:
            new_curves.append(new_curve)
        
        current_curve = remaining_curve
    
    if len(current_curve) > 0:
        new_curves.append(current_curve)
    
    return new_curves

def split_all_curves_by_curvature(curves, curvature_threshold=0.2, exclusion_distance=10, min_length=8, distance_threshold=30):
    all_new_curves = []
    
    for curve in curves:
        new_curves = split_curve_by_curvature(curve, curvature_threshold, exclusion_distance, min_length, distance_threshold)
        all_new_curves.extend(new_curves)
    
    return all_new_curves


In [30]:
def dfs(node, start_node, visited, path, adj_list, partner, unique_cycles):
    # Track the path of nodes
    path.append(node)
    visited.add(node)
    
    for neighbor, curve_number in adj_list[node]:
        
        if len(path) ==2 and partner[path[0]] == path[1] and euclidean_distance(umap[path[0]], umap[path[1]]) < 10:
            curve_path =[]
            curve_path.append(curve_num[path[0]])
            cycle_representation = frozenset(curve_path)
            if cycle_representation not in unique_cycles and len(cycle_representation) > 0:
                unique_cycles[cycle_representation] = list(path) + [start_node]
        
        if neighbor == start_node and len(path) > 2:
            # Generate curve path from node path
            curve_path = []
            for i in range(len(path)-1):
                cur_node = path[i]
                next_node = path[i + 1]
                if partner[cur_node] == next_node:
                    curve_path.append(curve_num[cur_node])
            if partner[path[len(path)-1]] == neighbor:
                    curve_path.append(curve_num[neighbor])
            
            cycle_representation = frozenset(curve_path)
            if cycle_representation not in unique_cycles and len(cycle_representation) > 0:
                unique_cycles[cycle_representation] = list(path) + [start_node]
                #print(f"Added new cycle: {unique_cycles[cycle_representation]}")
        elif neighbor not in visited:
            dfs(neighbor, start_node, visited, path, adj_list, partner, unique_cycles)

    path.pop()
    visited.remove(node)

def find_closed_curves(adj_list, num_nodes):
    unique_cycles = {}

    for start_node in range(num_nodes):
        #print(f"Starting new DFS from start_node={start_node}")
        visited = set()
        dfs(start_node, start_node, visited, [], adj_list, partner, unique_cycles)
       # print(f"Completed DFS for start_node={start_node}\n")

   # print(f"Total Unique Cycles Found: {len(unique_cycles)}")
    return unique_cycles



In [7]:
import numpy as np

def reconstruct_single_curve(node_sequence, umap, XY, partner):
    curve_points = []
    i = 0
    
    while i < len(node_sequence) - 1:
        cur_node = node_sequence[i]
        next_node = node_sequence[i + 1]
        
        # Check if next node is the partner of the current node
        if partner[cur_node] != next_node:
            # Directly add the point if it's a partner
            i += 1
        else:
            # Find a curve in XY that starts or ends with the points corresponding to cur_node and next_node
            start_point = umap[cur_node]
            end_point = umap[next_node]
            found_curve = False
            
            for curve in XY:
                curve_start = curve[0]
                curve_end = curve[-1]
                
                if (np.array_equal(curve_start, start_point) and np.array_equal(curve_end, end_point)) or (np.array_equal(curve_start, end_point) and np.array_equal(curve_end, start_point)):
                    # If start_point and end_point match, add points to the curve
                    if np.array_equal(curve_start, start_point):
                        curve_points.extend(curve)
                    else:
                        curve_points.extend(curve[::-1])
                    
                    found_curve = True
                    break
            
            if not found_curve:
                raise ValueError("Curve with the specified start and end points not found in XY.")
            
            # Move ahead to the next to next node
            i += 2
    
    return np.array(curve_points)

# Example usage
# node_sequence: the sequence of nodes in the closed curve
# umap: dictionary mapping node indices to points
# XY: list of curves where each curve is a sequence of points
# partner: dictionary mapping each node to its partner node



In [8]:
def distances_to_centroid(points):
    # Convert to numpy array for easier calculations
    points = np.array(points)
    
    # Calculate centroid
    centroid = points.mean(axis=0)
    
    # Compute distances to centroid
    distances = np.sqrt(((points - centroid) ** 2).sum(axis=1))
    
    return distances

In [20]:
def calc_curvature(points):
    points = np.array(points)
    num_points = len(points)
    
    # Calculate angles between segments
    curvatures = []
    for i in range(1, num_points - 1):
        p1 = points[i - 1]
        p2 = points[i]
        p3 = points[i + 1]
        
        # Vectors
        v1 = p2 - p1
        v2 = p3 - p2
        
        # Angle between vectors
        dot_product = np.dot(v1, v2)
        norm_v1 = np.linalg.norm(v1)
        norm_v2 = np.linalg.norm(v2)
         # Prevent division by zero
        if norm_v1 == 0 or norm_v2 == 0:
            curvatures.append(0.0)
            continue
        
        cos_angle = dot_product / (norm_v1 * norm_v2)
        
        # Clamp the cosine value to avoid invalid inputs for arccos
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        
        angle = np.arccos(cos_angle)
        curvatures.append(angle)
    
    curvatures = np.array(curvatures)
    padded_curvatures = np.concatenate(([0], curvatures, [0]))
    
    return padded_curvatures

In [21]:
max_len = 1000
def extract_features(points):
    # Calculate curvature vector
    curvatures = calc_curvature(points)
    
    # Calculate distance vector to centroid
    distances = distances_to_centroid(points)
    
    # Combine features into a single matrix
    # Ensure the feature vectors are of the same length
    if len(curvatures) != len(distances):
        min_len = min(len(curvatures), len(distances))
        curvatures = curvatures[:min_len]
        distances = distances[:min_len]
    
    features = np.array([[d, c] for d, c in zip(distances, curvatures)])
    
    if len(features) < max_len:
        # Pad with [-1, -1]
        padding_length = max_len - len(features)
        features_padded = np.vstack([features, np.full((padding_length, 2), -1)])
    else:
        # Truncate
        features_padded = features[:max_len]
    
    return features_padded

In [34]:
import numpy as np

def find_corners(points, angle_threshold=0.2, distance=5):
    def safe_arccos(x):
        return np.arccos(np.clip(x, -1.0, 1.0))
    
    def calculate_curvature(p1, p2, p3):
        v1 = np.array(p2) - np.array(p1)
        v2 = np.array(p3) - np.array(p2)
        dot_product = np.dot(v1, v2)
        norm_v1 = np.linalg.norm(v1)
        norm_v2 = np.linalg.norm(v2)
        if norm_v1 == 0 or norm_v2 == 0:
            return 0
        angle = safe_arccos(dot_product / (norm_v1 * norm_v2))
        return angle
    
    if not isinstance(points, np.ndarray):
        # Convert to NumPy array if it's not already
        points = points.to_numpy()
    
    num_points = len(points)
    corners = []
    marked_indices = set()
    
    i = 0
    loop_count = 0  # Counter to prevent infinite loop
    
    while loop_count < num_points:
        if i in marked_indices:
            i = (i + 1) % num_points
            loop_count += 1
            continue
        
        # Determine indices for curvature calculation with cyclic behavior
        p1 = points[i]
        p2 = points[(i + distance // 2) % num_points]
        p3 = points[(i + distance) % num_points]
        
        curvature = calculate_curvature(p1, p2, p3)
        if curvature > angle_threshold:
            corners.append(p2)
            # Mark neighbors to skip, handle cyclic behavior
            for j in range(-distance, distance + 1):
                marked_index = (i + j) % num_points
                marked_indices.add(marked_index)
            i = (i + distance) % num_points  # Move ahead to next to next point
        else:
            i = (i + 1) % num_points
        
        loop_count += 1
    
    return np.array(corners)

In [12]:
def calculate_dimensions(corners):
    def calculate_distance(p1, p2):
        return np.linalg.norm(np.array(p2) - np.array(p1))
    
    distances = []
    for j in range(len(corners) - 1):
        dist = calculate_distance(corners[j], corners[j + 1])
        distances.append(dist)

    # Alternate labeling as height and breadth
    heights = distances[::2]
    breadths = distances[1::2]

    avg_height = np.mean(heights) if heights else 0
    avg_breadth = np.mean(breadths) if breadths else 0

    return avg_height, avg_breadth

In [13]:
def generate_circle_points(center, radius, num_points=100):
    """
    Generate points on a circle in sequence at equal distances.
    
    Args:
    - center (tuple): The (x, y) coordinates of the circle's center.
    - radius (float): The radius of the circle.
    - num_points (int): The number of points to generate. Default is 100.
    
    Returns:
    - np.array: A numpy array of shape (num_points, 2) containing the (x, y) coordinates of the points.
    """
    h, k = center
    points = []

    # Increment angle by 2π/num_points for equal spacing
    delta_theta = 2 * np.pi / num_points

    for i in range(num_points):
        theta = i * delta_theta
        x = h + radius * np.cos(theta)
        y = k + radius * np.sin(theta)
        points.append((x, y))

    return np.array(points)

In [14]:
import numpy as np

def distance(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def calculate_slope(p1, p2):
    if p2[0] - p1[0] == 0:
        return float('inf')  # Vertical line
    return (p2[1] - p1[1]) / (p2[0] - p1[0])

def calculate_angle_between_lines(p1, p2, p3, p4):
    slope1 = calculate_slope(p1, p2)
    slope2 = calculate_slope(p3, p4)
    
    if slope1 == float('inf'):
        angle1 = np.pi / 2
    else:
        angle1 = np.arctan(slope1)
    
    if slope2 == float('inf'):
        angle2 = np.pi / 2
    else:
        angle2 = np.arctan(slope2)
    
    return abs(angle1 - angle2)

def rotate_points(points, angle):
    rotation_matrix = np.array([
        [np.cos(angle), -np.sin(angle)],
        [np.sin(angle), np.cos(angle)]
    ])
    return np.dot(points, rotation_matrix.T)

def adjust_to_perfect_rectangle(corners):
    # Convert to numpy array for easier manipulation
    corners = np.array(corners)
    
    # Calculate pairwise distances
    dists = {
        (0, 1): distance(corners[0], corners[1]),
        (1, 2): distance(corners[1], corners[2]),
        (2, 3): distance(corners[2], corners[3]),
        (3, 0): distance(corners[3], corners[0]),
        (0, 2): distance(corners[0], corners[2]),
        (1, 3): distance(corners[1], corners[3])
    }
    
    # Identify side lengths and diagonals
    side_lengths = sorted([dists[(0, 1)], dists[(1, 2)], dists[(2, 3)], dists[(3, 0)]])
    diagonals = sorted([dists[(0, 2)], dists[(1, 3)]])
    
    # Average lengths of sides that should be equal
    side1 = (side_lengths[0] + side_lengths[1]) / 2
    side2 = (side_lengths[2] + side_lengths[3]) / 2
    
    # Define new perfect rectangle corners based on these dimensions
    new_corners = np.array([
        [0, 0],
        [side1, 0],
        [side1, side2],
        [0, side2]
    ])
    
    # Translate new_corners to the centroid of the original corners
    centroid = np.mean(corners, axis=0)
    adjusted_corners = new_corners + (centroid - np.mean(new_corners, axis=0))
    
    # Calculate angles in original and adjusted rectangles
    original_angle1 = calculate_angle_between_lines(corners[0], corners[1], corners[1], corners[2])
    original_angle2 = calculate_angle_between_lines(corners[1], corners[2], corners[2], corners[3])
    
    adjusted_angle1 = calculate_angle_between_lines(adjusted_corners[0], adjusted_corners[1], adjusted_corners[1], adjusted_corners[2])
    adjusted_angle2 = calculate_angle_between_lines(adjusted_corners[1], adjusted_corners[2], adjusted_corners[2], adjusted_corners[3])
    
    # Calculate the average angle difference
    angle_diff1 = np.degrees(abs(original_angle1 - adjusted_angle1))
    angle_diff2 = np.degrees(abs(original_angle2 - adjusted_angle2))
    
    # Decide whether to rotate based on angle differences
    if max(angle_diff1, angle_diff2) > 10:
        # Rotate the adjusted points to match the original orientation
        angle = (original_angle1 - adjusted_angle1 + np.pi / 2) % (2 * np.pi) - np.pi
        rotated_corners = rotate_points(adjusted_corners, angle)
    else:
        rotated_corners = adjusted_corners
    
    return rotated_corners.tolist()

In [15]:
import numpy as np

def cal_cur(center, side1, side2):
    center = np.array(center)
    side1 = np.array(side1)
    side2 = np.array(side2)

    vector1 = center - side1
    vector2 = side2 - center
    
    #print(vector1, vector2)

    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)

    # Check for zero-length vectors
    if norm1 == 0 or norm2 == 0:
        return 50  # or handle appropriately

    # Calculate angle between vectors
    cos_angle = np.dot(vector1, vector2) / (norm1 * norm2)
    cos_angle = np.clip(cos_angle, -1.0, 1.0)  # Ensure valid range for arccos

    angle = np.arccos(cos_angle)

    # Convert angle to degrees
    return np.degrees(angle)


In [32]:
def dfs_find_open_paths(node, parent, visited, path, curves_in_path, adj_list, unique_paths, umap, exclusion_distance = 10):
    path.append(node)
    visited.add(node)

    is_end = True  # To check if it's an endpoint

    for neighbor, curve_number in adj_list[node]:
        if neighbor != parent:
            is_end = False

            # Get the points for the current node, its partner, and the neighbor's partner
            if neighbor != partner[node]:
                center_point = umap[node]
                partner_point = umap[partner[node]]
                neighbor_start = umap[partner[neighbor]]

                # Calculate the curvature at the center point
                curvature = cal_cur(center_point, partner_point, neighbor_start)

                #print(center_point, partner_point, neighbor_start, curvature)
            else:
                curvature =0
            # If the curvature is less than 20 degrees, move to this neighbor
            if curvature < 20.0:
                if neighbor not in visited:
                    dfs_find_open_paths(neighbor, node, visited, path, curves_in_path + [curve_number], adj_list, unique_paths, umap, exclusion_distance)

    # Check if the node is an endpoint (no other neighbors except its parent)
    if is_end and parent is not None and euclidean_distance(umap[parent], umap[node]) >= exclusion_distance:
        path_representation = frozenset(curves_in_path)
        if path_representation not in unique_paths:
            unique_paths[path_representation] = list(path)

    path.pop()
    visited.remove(node)


In [31]:
def find_open_curves(adj_list, num_nodes, umap, exclusion_distance=10):
    unique_paths = {}

    for start_node in range(num_nodes):
        visited = set()
        if len(adj_list[start_node]) == 1:
            dfs_find_open_paths(start_node, None, visited, [], [], adj_list, unique_paths, umap, exclusion_distance)

    return unique_paths


In [2]:
import numpy as np

def normalize_and_scale(points, grid_size):
    min_x, max_x = np.min(points[:, 0]), np.max(points[:, 0])
    min_y, max_y = np.min(points[:, 1]), np.max(points[:, 1])
    
    # Normalize coordinates to [0, 1]
    normalized_x = (points[:, 0] - min_x) / (max_x - min_x)
    normalized_y = (points[:, 1] - min_y) / (max_y - min_y)
    
    # Scale to fit into the grid
    scaled_x = (normalized_x * (grid_size - 1)).astype(int)
    scaled_y = (normalized_y * (grid_size - 1)).astype(int)
    
    return np.stack([scaled_x, scaled_y], axis=1)

def points_to_image(points, grid_size):
    image = np.zeros((grid_size, grid_size), dtype=np.float32)
    for x, y in points:
        image[y, x] = 1  # Set the pixel to 1
    return image

def convert_points_to_image(points, grid_size):
    scaled_points = normalize_and_scale(points, grid_size)
    image = points_to_image(scaled_points, grid_size)
    return image


In [2]:
import numpy as np

def calculate_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def calculate_average_side_length(corners):
    n = len(corners)
    if n < 2:
        return 0.0
    total_length = 0.0
    for i in range(n):
        total_length += calculate_distance(corners[i], corners[(i + 1) % n])
    return total_length / n

def find_next_point_on_circle(center, radius, prev_point, side_length, angle_increment=0.01):
    # Find the next point on the circle in a fixed direction (clockwise)
    angle = 0
    while True:
        next_point = center + radius * np.array([np.cos(angle), np.sin(angle)])
        if np.isclose(calculate_distance(prev_point, next_point), side_length, atol=1e-2):
            return next_point
        angle += angle_increment  # Increment angle to find a suitable point

def generate_polygon_points_on_circle(corners, centroid):
    n = len(corners)
    if n < 3:
        return 0;
    
    # Step 1: Calculate the average side length
    average_side_length = calculate_average_side_length(corners)
    
    # Step 2: Calculate the radius of the circle using the distance from the centroid to the first corner
    radius = calculate_distance(centroid, corners[0])
    
    # Initialize the list of points with the first corner
    points = [corners[0]]
    
    # Generate the remaining points
    for _ in range(1, n):
        next_point = find_next_point_on_circle(centroid, radius, points[-1], average_side_length)
        points.append(next_point)
    
    return np.array(points)

In [25]:
import numpy as np

def remove_close_corners(corners, min_distance=10.0):
    if len(corners) < 2:
        return corners  # No need to process if less than 2 points
    
    filtered_corners = [corners[0]]  # Start with the first corner
    
    for i in range(1, len(corners)):
        distance = np.linalg.norm(corners[i] - filtered_corners[-1])
        if distance >= min_distance:
            filtered_corners.append(corners[i])
    
    return np.array(filtered_corners)

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean

def calculate_center(x, y):
    # Calculate the centroid of the points
    return np.mean(x), np.mean(y)

def calculate_ellipse_parameters(x, y):
    # Calculate the centroid
    center = calculate_center(x, y)
    
    # Calculate the distance between all pairs of points
    max_distance = 0
    max_pair = (0, 0)
    N = len(x)
    for i in range(N):
        for j in range(i + 1, N):
            distance = euclidean((x[i], y[i]), (x[j], y[j]))
            if distance > max_distance:
                max_distance = distance
                max_pair = (i, j)
    
    semi_major_axis = max_distance / 2
    
    # Calculate the slope of the major axis
    x1, y1 = x[max_pair[0]], y[max_pair[0]]
    x2, y2 = x[max_pair[1]], y[max_pair[1]]
    slope = np.arctan2(y2 - y1, x2 - x1)
    
    # Calculate the semi-minor axis
    midpoint_index = N // 2
    min_distance = np.inf
    for i in range(N):
        j = (i + midpoint_index) % N
        distance = euclidean((x[i], y[i]), (x[j], y[j]))
        if distance < min_distance:
            min_distance = distance
    
    semi_minor_axis = min_distance / 2
    
    return center, semi_major_axis, semi_minor_axis, slope

def generate_ellipse(center, a, b, slope, num_points=100):
    x0, y0 = center
    t = np.linspace(0, 2 * np.pi, num_points)
    
    cos_theta = np.cos(slope)
    sin_theta = np.sin(slope)
    
    xt = a * np.cos(t)
    yt = b * np.sin(t)
    
    x = x0 + cos_theta * xt - sin_theta * yt
    y = y0 + sin_theta * xt + cos_theta * yt
    
    return np.column_stack((x, y))

